In [201]:
%matplotlib inline
import matplotlib.pyplot as plt 
from IPython.display import clear_output

In [202]:
#!/usr/bin/env python
# coding: utf-8

"""CMOS MT9V011 Cluster Detection"""

#Import libraries
import os
import sys
import fnmatch
import pickle
import pandas as pd
from math import sqrt
from tqdm import tqdm
from skimage.feature import blob_log

import numpy as np
from numpy import interp
np.version.version


'1.17.3'

In [301]:
#Files directory
DATADIR = "/nfs/NASPG/BTData/Jul2012_CMOS_data/MT9V011_Firenze_2012_07_13/TEST/"
#DATADIR = "/nfs/NASPG/BTData/Jul2012_CMOS_data/MT9V011_Firenze_2012_07_13/16_MT9V011_017_G01_050ms_3MeV_-60/"
#DATADIR = sys.argv[1]

PEDS = np.zeros((307200,1))
PEDS[: , :] = np.nan

CLUSTERS = pd.DataFrame({"Max":[], "Sum":[], "Size":[], "row":[],"column":[]})


In [280]:
#Evaluate threshold for pedestal calculation
if not os.path.isfile(DATADIR + "pedestals.npy"):
    RAW_VALUES = np.array(0)
    for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR),
                                                '*.txt'))):
        clear_output(wait=True)
        print("Processing " + _filename + " for threshold computation")
        RAW_VALUES = np.append(RAW_VALUES, np.loadtxt(DATADIR + _filename, dtype="int"))
        THRESH = (RAW_VALUES.mean() + 2 * RAW_VALUES.std()).astype(int)


Processing 16_MT9V011_017_G01_050ms_3MeV_-60_00002_fr0000.txt for threshold computation


100%|██████████| 3/3 [00:00<00:00,  4.18it/s]


In [281]:
im = np.loadtxt(DATADIR + _filename)
im = im.reshape(307200,1)
if im.shape == (307200,1):
    mask = im < THRESH  # mask of pixels under threshold (good for pedestals)\
    im[im >= THRESH] = np.nan  # to prevent adding events with signal
    PEDS = np.concatenate((PEDS, im), axis=1)

In [278]:
PEDS.shape

(307200, 2)

In [303]:
#Calculate or (load) Pedestals
if not os.path.isfile(DATADIR + "pedestals.npy"):
    for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR),
                                                '*.txt'))):
        clear_output(wait=True)
        print("Processing " + _filename + " for pedestals computation")
        im = np.loadtxt(DATADIR + _filename)
        if im.shape == (307200,):
            mask = im < THRESH  # mask of pixels under threshold (good for pedestals)
            im[im >= THRESH] = np.nan  # to prevent adding events with signal
            im = np.atleast_2d(im).T
            PEDS = np.concatenate((PEDS, im), axis=1)
        else:
            continue
    #PEDS = np.nanmean(PEDS,axis=1)
    PEDS.dump(DATADIR + "pedestals.npy")
else:
    PEDS = np.load(DATADIR + "pedestals.npy", allow_pickle="True")

In [310]:
np.nanmean(PEDS,axis=1).shape

/home/gsilvest/anaconda3/envs/gigi/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


(307200,)

In [306]:
#Find clusters in every frame
for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR), '*.txt'))):
    print("Processing " + _filename)
    frame = np.loadtxt(DATADIR + _filename, dtype="int")

    if frame.shape == PEDS.shape:
        frame = frame - PEDS
    else:
        continue

    frame = frame.reshape(480, 640)
    _temp = interp(frame, [0, 1024], [0, 1])
    
    #Find "blobs"
    blobs_log = blob_log(_temp, max_sigma=50, num_sigma=50, threshold=.05)
    blobs_log[:, 2] = blobs_log[:, 2] * 2 * sqrt(2)
    
    for _x, _y, _r in blobs_log:
        _min_y = (_y - _r).astype(int)
        _max_y = (_y + _r).astype(int)
        _min_x = (_x - _r).astype(int)
        _max_x = (_x + _r).astype(int)
        _subim = frame[_min_x:_max_x, _min_y:_max_y]
        CLUSTERS = CLUSTERS.append({'Max': np.max(_subim), 'Sum' : np.sum(_subim),
                                    'Size' : _subim.size,
                                    'row' : _x,
                                    'column' : _y},
                                   ignore_index=True)
    
    CLUSTERS.to_feather(DATADIR + "clusters.feather")

  0%|          | 0/3 [00:00<?, ?it/s]

Processing 16_MT9V011_017_G01_050ms_3MeV_-60_00000_fr0000.txt


 33%|███▎      | 1/3 [00:00<00:00,  4.38it/s]

Processing 16_MT9V011_017_G01_050ms_3MeV_-60_00001_fr0000.txt


 67%|██████▋   | 2/3 [00:00<00:00,  4.44it/s]

Processing 16_MT9V011_017_G01_050ms_3MeV_-60_00002_fr0000.txt


100%|██████████| 3/3 [00:00<00:00,  4.50it/s]
